In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau 
from keras.preprocessing.image import ImageDataGenerator

from keras.optimizers import RMSprop, SGD


In [ ]:
os.environ['KAGGLE_USERNAME'] = "mratel" # username from the json file
os.environ['KAGGLE_KEY'] = "" # key from the json file

!kaggle competitions download -c digit-recognizer

train = pd.read_csv('train.csv.zip')
test = pd.read_csv('test.csv.zip')

y_train = train["label"]
X_train = train.drop(labels="label", axis=1)

#sns.countplot(y_train)
#X_train.isnull().any().describe()
#test.isnull().any().describe()

# Normalisation
X_train = X_train / 255
test = test / 255

#Reshape pour convolution
X_train1 = X_train.values.reshape(-1,28,28,1)
test1 = test.values.reshape(-1,28,28,1)

#Encodage label
y_train = to_categorical(y_train, num_classes = 10)

#Split apprentissage/validation
X_app, X_val, y_app, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=2)

#Split apprentissage/validation convolution
X_app1, X_val1, y_app, y_val = train_test_split(X_train1, y_train, test_size=0.1, random_state=2, stratify=y_train)

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

#g = plt.imshow(X_train1[3][:,:,0])

sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


# NN

In [ ]:
model = Sequential()

model.add(Dense(800, input_shape=(784,)))
model.add(Activation("relu"))
model.add(Dense(units=10))
model.add(Activation("softmax"))

optimizer1 = SGD(lr=0.001, momentum=0.0, nesterov=False)
optimizer2 = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(optimizer = optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X_app, y_app, validation_data = (X_val, y_val), epochs=10, batch_size=86)

In [ ]:
pred = model.predict(test)

pred = np.argmax(pred, axis = 1)

pred = pd.Series(pred, name="Label")

submission = pd.concat([pd.Series(range(1,28001), name = "ImageId"),pred],axis = 1)

submission.to_csv("submission_mnist.csv", index=False)

In [ ]:
model.predict(test)

# Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_app1)

# Small CNN 

In [ ]:
model = Sequential()

model.add(Conv2D(filters=64,kernel_size=(3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(filters=32,kernel_size=(3,3), activation='relu'))
model.add(Flatten())

model.add(Dense(units=10))
model.add(Activation("softmax"))

model.compile(optimizer = optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X_app1, y_app, validation_data = (X_val1, y_val), epochs=10, batch_size=32, verbose=1)

In [ ]:
pred = model.predict(test1)

pred = np.argmax(pred, axis = 1)

pred = pd.Series(pred, name="Label")

submission = pd.concat([pd.Series(range(1,28001), name = "ImageId"),pred],axis = 1)

submission.to_csv("submission_mnist.csv", index=False)

# Medium CNN

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation = "relu"))
model.add(Dense(10, activation = "softmax"))

model.compile(optimizer = optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

es = [EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2), ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

model.fit(X_app1, y_app, validation_data = (X_val1, y_val), epochs=10, batch_size=32, verbose=1, callbacks=es)

In [ ]:
#On charge le meilleur modèle enregistré
model = load_model('best_model.h5')

pred = model.predict(test1)

pred = np.argmax(pred, axis = 1)

pred = pd.Series(pred, name="Label")

submission = pd.concat([pd.Series(range(1,28001), name = "ImageId"),pred],axis = 1)

submission.to_csv("submission_mnist.csv", index=False)

# Large CNN

In [ ]:
batch_size=32

model = Sequential()

model.add(Conv2D(filters = 30, kernel_size = (5,5), activation ='relu', input_shape = (28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 15, kernel_size = (3,3), activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation = "relu"))
model.add(Dense(50, activation = "relu"))
model.add(Dense(10, activation = "softmax"))

model.compile(optimizer = 'adam', loss="categorical_crossentropy", metrics=["accuracy"])

es = [EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2), ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

model.fit_generator(datagen.flow(X_app1, y_app, batch_size=batch_size), validation_data = (X_val1, y_val), epochs=30, steps_per_epoch=X_app1.shape[0] / batch_size, verbose=1, callbacks=es)

Epoch 1/30
1182/1181 [==============================] - 36s 31ms/step - loss: 0.4432 - acc: 0.8547 - val_loss: 0.0837 - val_acc: 0.9755
Epoch 2/30
1182/1181 [==============================] - 36s 31ms/step - loss: 0.1680 - acc: 0.9480 - val_loss: 0.0677 - val_acc: 0.9802
Epoch 3/30
1182/1181 [==============================] - 36s 31ms/step - loss: 0.1290 - acc: 0.9589 - val_loss: 0.0569 - val_acc: 0.9831
Epoch 4/30
1182/1181 [==============================] - 36s 30ms/step - loss: 0.1110 - acc: 0.9656 - val_loss: 0.0511 - val_acc: 0.9850
Epoch 5/30
1182/1181 [==============================] - 35s 30ms/step - loss: 0.1003 - acc: 0.9689 - val_loss: 0.0451 - val_acc: 0.9864
Epoch 6/30
1182/1181 [==============================] - 35s 30ms/step - loss: 0.0888 - acc: 0.9718 - val_loss: 0.0387 - val_acc: 0.9881
Epoch 7/30
1182/1181 [==============================] - 36s 30ms/step - loss: 0.0833 - acc: 0.9749 - val_loss: 0.0305 - val_acc: 0.9902
Epoch 8/30
1182/1181 [==========================

In [ ]:
#On charge le meilleur modèle enregistré
model = load_model('best_model.h5')

pred = model.predict(test1)

pred = np.argmax(pred, axis = 1)

pred = pd.Series(pred, name="Label")

submission = pd.concat([pd.Series(range(1,28001), name = "ImageId"),pred],axis = 1)

submission.to_csv("largeCNN.csv", index=False)

# CNN ghouzam

In [ ]:
# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

# Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

batch_size = 86

es = [EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=300), ModelCheckpoint(filepath='best_model.h5', monitor='val_acc', save_best_only=True), learning_rate_reduction]

model.fit_generator(datagen.flow(X_app1, y_app, batch_size=batch_size),
                              epochs = 30, validation_data = (X_val1,y_val),
                              verbose = 1, steps_per_epoch=X_train.shape[0] / batch_size
                              , callbacks=es)

Epoch 1/30
489/488 [==============================] - 297s 608ms/step - loss: 0.4132 - acc: 0.8700 - val_loss: 0.0579 - val_acc: 0.9795
Epoch 2/30
489/488 [==============================] - 293s 599ms/step - loss: 0.1524 - acc: 0.9559 - val_loss: 0.0495 - val_acc: 0.9855
Epoch 3/30
489/488 [==============================] - 293s 598ms/step - loss: 0.1320 - acc: 0.9641 - val_loss: 0.0443 - val_acc: 0.9886
Epoch 4/30
489/488 [==============================] - 293s 600ms/step - loss: 0.1259 - acc: 0.9675 - val_loss: 0.0581 - val_acc: 0.9848
Epoch 5/30
489/488 [==============================] - 295s 603ms/step - loss: 0.1252 - acc: 0.9686 - val_loss: 0.0628 - val_acc: 0.9860
Epoch 6/30
489/488 [==============================] - 293s 600ms/step - loss: 0.1309 - acc: 0.9702 - val_loss: 0.0426 - val_acc: 0.9864

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 7/30
489/488 [==============================] - 294s 600ms/step - loss: 0.0828 - acc: 0.9782 - va

In [ ]:
#On charge le meilleur modèle enregistré
model = load_model('best_model.h5')

pred = model.predict(test1)

pred = np.argmax(pred, axis = 1)

pred = pd.Series(pred, name="Label")

submission = pd.concat([pd.Series(range(1,28001), name = "ImageId"),pred],axis = 1)

submission.to_csv("largeCNN.csv", index=False)